# 3.4 Transitive Closure & Recursion

Recursive rules are self referential to some degree, meaning the facts the infer may create the conditions for additional facts to be inferred.

This iterative cycle will continue until the body conditions of the rules involved are no longer met.

Recursion is the process used in identifying property paths. The rules make their way along the path, incorporating one new link at a time.

A common use of recursion, transitive closure directly connects all nodes that are joined by any repeating pattern downstream, be that a chain of the same relationships or a more complex shape.



## Example

The following example shows how recursion can be used to traverse a graph network, here a collection  of interconnected railway stations.

The aim is to uncover stations that are indirectly accessible, joined by a series of directly connected stations.

In [5]:
tc_data = """
@prefix : <https://rdfox.com/example#> .

:webServer a :WebServer ;
    :dependsOn :machine .

:machine a :StorageMachine ;
    :dependsOn :network ,
        :powerSupply .

:network a :Network .

:powerSupply a :PowerSupply .

"""

In [18]:
tc_rules = """

[?asset1, :dependsViaClosure, ?asset2] :-
   [?asset1, :dependsOn, ?asset2] .

[?asset1, :dependsViaClosure, ?asset3] :-
   [?asset1, :dependsViaClosure, ?asset2],
   [?asset2, :dependsOn, ?asset3] .

"""

### Writing recursive rule sets

The first rule above is note recursive, it acts as an anchor that provides stable starting conditions.

The second is the recursive rules. It provides conditions for continual looping able to iterate again and again as new data is added by the previous iteration.

While not every application uses this setups, it is common for recursive rule sets to contain an anchor as it restricts inferences to just those that are determined necessary. Recursion has the potential to rapidly expand the size of the dataset if not properly contained.

In [19]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=tc_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=tc_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/3_3-TransitiveClosureQuery.rq", "r") as file:
    tc_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": tc_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Assets whose function is dependent on others ===')
print(response.text)


=== Assets whose function is dependent on others ===
?upstreamAsset	?downstreamAsset
<https://rdfox.com/example#webServer>	<https://rdfox.com/example#machine>
<https://rdfox.com/example#webServer>	<https://rdfox.com/example#network>
<https://rdfox.com/example#webServer>	<https://rdfox.com/example#powerSupply>
<https://rdfox.com/example#machine>	<https://rdfox.com/example#network>
<https://rdfox.com/example#machine>	<https://rdfox.com/example#powerSupply>



### End entities

It can be valuable to know the first or last member of a chain and with negation (see 2.2) this becomes trivial - we simply look for there not to exist a proceeding or succeeding member of the chain respectively.

Below is a rule that identifies the web server as a top level asset.

In [10]:
last_entity_rule = """

[?asset1, a, :TopLevelAsset] :-
    [?asset1, :dependsOn, ?asset2] 
    NOT EXISTS ?asset0 IN ( [?asset0, :dependsOn, ?asset1] ).

"""

### Cyclic relationships

Recursion can create cyclic relationships, either intentionally or otherwise, so it can be important to know when and where they exist.

Cyclic relationships that have been created through recursion are very simple to detect - we can simply look for an entity that belongs to its own chain.

Below is a rule that does just that.

In [15]:

cyclic_data = """

:network :dependsOn :webServer .

"""

cyclic_rule = """

[?asset, a, :CyclicAsset] :-
    [?asset, :dependsViaClosure, ?asset].

"""

cyclic_query = """

SELECT ?cyclicAsset ?downstreamAsset
WHERE {
    ?cyclicAsset a :CyclicAsset ;
        :dependsOn ?downstreamAsset .
    
    ?downstreamAsset a :CyclicAsset .
} ORDER BY ASC(?cyclicAsset)

"""

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=cyclic_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Add rules
with open("../rules/3_4-PropertyPathsRules.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()
response = requests.post(rdfox_server + "/datastores/default/content", data=cyclic_rule)
assert_response_ok(response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": cyclic_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Cycles ===')
print(response.text)



=== Cycles ===
?cyclicAsset	?downstreamAsset
<https://rdfox.com/example#machine>	<https://rdfox.com/example#network>
<https://rdfox.com/example#network>	<https://rdfox.com/example#webServer>
<https://rdfox.com/example#webServer>	<https://rdfox.com/example#machine>



## Exercise

Complete the rule `3_3-TransitiveClosureRules.dlog` in the `rules` folder so that the query below can be used to directly find the transitive dependencies of all primary assets - that is, assets that have a `:backupPriority` of 0.

In [25]:
tc_sparql = """

SELECT ?upstreamAsset ?downstreamAsset
WHERE {
    ?upstreamAsset :hasPrimaryDependency ?downstreamAsset
} ORDER BY DESC(?upstreamAsset) ASC(?downstreamAsset)

"""

Here is a representative sample of the data in `2_3-BlankNodesData.ttl`.

In [23]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:webServer1 a :WebServer;
    :backupPriority 0;
    :dependsOn :machine1 ;
    :hasBackup :webServer2 .

:webServer2 a :WebServer;
    :backupPriority 1;
    :dependsOn :machine2 .

:machine1 a :ServerMachine;
    :backupPriority 0;
    :dependsOn :network1, :powerSupply1 .

:machine2 a :ServerMachine;
    :backupPriority 1;
    :dependsOn :network2, :powerSupply2 .

"""

### Check your work

Run the query below to verify the results.

In [29]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/3_3-TransitiveClosureData.ttl", "r") as file:
    tc_data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=tc_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/3_3-TransitiveClosureRules.dlog", "r") as file:
    tc_rules = file.read()
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=tc_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": tc_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Primary Assets Transitive Dependencies ===')
print(response.text)


=== Primary Assets Transitive Dependencies ===
?upstreamAsset	?downstreamAsset
<https://rdfox.com/example/webServerService>	<https://rdfox.com/example/machine1>
<https://rdfox.com/example/webServerService>	<https://rdfox.com/example/network1>
<https://rdfox.com/example/webServerService>	<https://rdfox.com/example/powerSupply1>
<https://rdfox.com/example/webServerService>	<https://rdfox.com/example/webServer1>
<https://rdfox.com/example/webServer1>	<https://rdfox.com/example/machine1>
<https://rdfox.com/example/webServer1>	<https://rdfox.com/example/network1>
<https://rdfox.com/example/webServer1>	<https://rdfox.com/example/powerSupply1>
<https://rdfox.com/example/onlineStoreSystem>	<https://rdfox.com/example/machine1>
<https://rdfox.com/example/onlineStoreSystem>	<https://rdfox.com/example/machine5>
<https://rdfox.com/example/onlineStoreSystem>	<https://rdfox.com/example/machine9>
<https://rdfox.com/example/onlineStoreSystem>	<https://rdfox.com/example/network1>
<https://rdfox.com/exa

## Visualise the results

These power in these results is much clearer when visualised.

Open them in the RDFox Explorer [here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FupstreamAsset%20%3FdownstreamAsset%0AWHERE%20%7B%0A%20%20%20%20%3FupstreamAsset%20%3AhasPrimaryDependency%20%3FdownstreamAsset%0A%7D%20ORDER%20BY%20DESC%28%3FupstreamAsset%29%20ASC%28%3FdownstreamAsset%29) and **change the layout to Circule**.

### You should see...

=== Primary Assets Transitive Dependencies ===
|?upstreamAsset|?downstreamAsset|
|-----------|-------------|
|<https://rdfox.com/example/webServerService>|	<https://rdfox.com/example/machine1>|
|<https://rdfox.com/example/webServerService>|	<https://rdfox.com/example/network1>|
|<https://rdfox.com/example/webServerService>|	<https://rdfox.com/example/powerSupply1>|
|<https://rdfox.com/example/webServerService>|	<https://rdfox.com/example/webServer1>|
|<https://rdfox.com/example/webServer1>|	<https://rdfox.com/example/machine1>|
|<https://rdfox.com/example/webServer1>|	<https://rdfox.com/example/network1>|
|<https://rdfox.com/example/webServer1>|	<https://rdfox.com/example/powerSupply1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/machine1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/machine5>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/machine9>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/network1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/network5>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/network9>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/onlineStoreApplicationServer1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/onlineStoreApplicationService>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/onlineStoreDataBase1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/onlineStoreDatabaseService>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/paymentProcessingService>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/powerSupply1>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/powerSupply5>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/powerSupply9>|
|<https://rdfox.com/example/onlineStoreSystem>|	<https://rdfox.com/example/webServer1>|
|...||
|<https://rdfox.com/example/machine5>|	<https://rdfox.com/example/powerSupply5>|
|<https://rdfox.com/example/machine1>|	<https://rdfox.com/example/network1>|
|<https://rdfox.com/example/machine1>|	<https://rdfox.com/example/powerSupply1>|